In [1]:
import pandas as pd
import numpy as np
import csv
import datetime 
import time
import json 
import random

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
DATA_ROOT_PATH = "/home/trongnhat/Documents/ITE_journal/Tuc_dataset/"

In [3]:
retail_rocket_root_path = DATA_ROOT_PATH + 'retail_rocket/' 
recobell_root_path = DATA_ROOT_PATH + "recobell/"
ml_1m_root_path = DATA_ROOT_PATH + "ml-1m/"

# Note:
    - Trong quá trình tạo dữ liệu dạng chuỗi để training, phát hiện ra cần thay đổi cách mã hóa dữ liệu addtocart và transaction
    - Trước hết, kiểm tra lại tại sao bất kì tương tác nào đều được thêm vào file implicit nhưng bây giờ đọc lại file lại không thấy?

# Read the filtered implicit and explicit data, combine and sort by time_stamp
## Note: 
    - Implicit and explicit with identical time_stamp will be merged to only one interaction
    - For a user, implicit and explicit of same item that happened one right after one (no interation with other item between this implicit and explicit) will be merge to only one interation, too

In [7]:
clean_implicit_data = pd.read_csv(retail_rocket_root_path + '_implicit.clean.txt', header=None, names=['uid', 'itemid', 'time', "label"])
clean_explicit_data = pd.read_csv(retail_rocket_root_path + '_explicit.clean.txt', header=None, names=['uid', 'itemid', 'time', "label"])

In [8]:
filter_implicit_data[:20]

,uid,itemid,time,label
0,257597,355908,1433221332117,1
1,992329,248676,1433224214164,1
2,951259,367447,1433221337106,1
3,810725,443030,1433221923240,1
4,794181,439202,1433223291897,1
5,824915,428805,1433220899221,1
6,339335,82389,1433221204592,1
7,176446,10572,1433222162373,1
8,598426,156489,1433223697356,1
9,223343,402625,1433224078165,1


In [45]:
with open(retail_rocket_root_path + 'i2index.txt') as f:
    active_item_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print(len(active_item_list))

83274


In [46]:
with open(retail_rocket_root_path + 'u2index.txt') as f:
    active_user_dict = {int(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print(len(active_user_list))

36751


In [22]:
print("Length before filter: ", len(clean_implicit_data))
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
clean_implicit_data = clean_implicit_data[clean_implicit_data.uid.isin(active_user_list)]
print("length after filter user: ", len(clean_implicit_data))
clean_implicit_data = clean_implicit_data[clean_implicit_data.itemid.isin(active_item_list)]
print("length after filter item: ", len(clean_implicit_data))

NameError: name 'clean_implicit_data' is not defined

In [12]:
print("Length before filter: ", len(clean_explicit_data))
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
clean_explicit_data = clean_explicit_data[clean_explicit_data.uid.isin(active_user_list)]
print("length after filter user: ", len(clean_explicit_data))
clean_explicit_data = clean_explicit_data[clean_explicit_data.itemid.isin(active_item_list)]
print("length after filter item: ", len(clean_explicit_data))

Length before filter:  88886
length after filter user:  39942
length after filter item:  39939


In [13]:
filter_implicit_data = clean_implicit_data.apply(lambda x: pd.Series([active_user_dict[x['uid']], active_item_dict[x['itemid']], x['time'], x["label"]], index=["uid", "itemid", "time", "label"]), axis=1)

In [14]:
filter_implicit_data

,uid,itemid,time,label
1,0,0,1433224214164,1
5,1,1,1433220899221,1
13,2,2,1433224070841,1
16,3,3,1433223203944,1
18,4,4,1433222147345,1
21,5,5,1433221380636,1
22,6,6,1433223176926,1
29,7,7,1433223146605,1
45,8,8,1433224041294,1
46,9,9,1433223076406,1


In [15]:
filter_explicit_data = clean_explicit_data.apply(lambda x: pd.Series([active_user_dict[x['uid']], active_item_dict[x['itemid']], x['time'], x["label"]], index=["uid", "itemid", "time", "label"]), axis=1)

In [16]:
filter_explicit_data

,uid,itemid,time,label
6,31,59933,1433193500981,3
9,136,2915,1433218757551,2
14,62,91,1433217911121,2
16,50,3167,1433174825959,2
17,90,152,1433175546693,2
20,50,840,1433174892472,2
25,103,26,1433174518180,3
26,118,187,1433176320539,2
27,103,26,1433174341347,2
28,245,765,1433175895575,2


In [17]:
filter_implicit_data.to_csv(retail_rocket_root_path + "_implicit.filter.clean.txt", header=False, index=False)

In [18]:
filter_explicit_data.to_csv(retail_rocket_root_path + "_explicit.filter.clean.txt", header=False, index=False)

In [20]:
filter_implicit_data = pd.read_csv(retail_rocket_root_path + "_implicit.filter.clean.txt", header=None, names=['uid', 'itemid', 'time', "label"])
filter_implicit_data

,uid,itemid,time,label
0,0,0,1433224214164,1
1,1,1,1433220899221,1
2,2,2,1433224070841,1
3,3,3,1433223203944,1
4,4,4,1433222147345,1
5,5,5,1433221380636,1
6,6,6,1433223176926,1
7,7,7,1433223146605,1
8,8,8,1433224041294,1
9,9,9,1433223076406,1


In [21]:
filter_explicit_data = pd.read_csv(retail_rocket_root_path + "_explicit.filter.clean.txt", header=None, names=['uid', 'itemid', 'time', "label"])
filter_explicit_data

,uid,itemid,time,label
0,31,59933,1433193500981,3
1,136,2915,1433218757551,2
2,62,91,1433217911121,2
3,50,3167,1433174825959,2
4,90,152,1433175546693,2
5,50,840,1433174892472,2
6,103,26,1433174518180,3
7,118,187,1433176320539,2
8,103,26,1433174341347,2
9,245,765,1433175895575,2


In [22]:
filtered_data = filter_implicit_data.append(filter_explicit_data)
filtered_data

,uid,itemid,time,label
0,0,0,1433224214164,1
1,1,1,1433220899221,1
2,2,2,1433224070841,1
3,3,3,1433223203944,1
4,4,4,1433222147345,1
5,5,5,1433221380636,1
6,6,6,1433223176926,1
7,7,7,1433223146605,1
8,8,8,1433224041294,1
9,9,9,1433223076406,1


In [23]:
sorted_filtered_pd_df = filtered_data.sort_values(by=["uid", "time"], ascending=True).reset_index(drop=True)
sorted_filtered_pd_df

,uid,itemid,time,label
0,0,20133,1430802384830,1
1,0,919,1432781848232,1
2,0,4244,1432782322169,1
3,0,0,1433224214164,1
4,0,2921,1433224672007,1
5,0,5692,1433225555976,1
6,0,6353,1433395087236,1
7,0,6353,1433395158782,1
8,0,6353,1433395205712,1
9,0,10195,1433480149682,1


In [25]:
sorted_filtered_pd_df.loc[sorted_filtered_pd_df.uid == 4]

,uid,itemid,time,label
65,4,14,1433221123646,1
66,4,2504,1433221805923,1
67,4,2504,1433221877110,1
68,4,4,1433221919130,1
69,4,1364,1433221969799,1
70,4,14,1433222104658,2
71,4,4,1433222132534,2
72,4,4,1433222147345,1
73,4,14,1433222234895,1
74,4,4,1433222272880,1


In [26]:
sorted_filtered_pd_df.to_csv(retail_rocket_root_path + "sorted_filtered_data", header=False, index=False)

In [20]:
retail_event_data = pd.read_csv("/home/trongnhat/Documents/Project/input/site_data/retail_rocket/raw_data/new_events.csv", header=0, names=["time", 'uid', "event", 'itemid', 'trans_id'])

In [21]:
retail_event_data

,time,uid,event,itemid,trans_id
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221337106,951259,view,367447,NaN
3,1433221923240,810725,view,443030,NaN
4,1433223291897,794181,view,439202,NaN
5,1433220899221,824915,view,428805,NaN
6,1433221204592,339335,view,82389,NaN
7,1433222162373,176446,view,10572,NaN
8,1433223697356,598426,view,156489,NaN
9,1433224078165,223343,view,402625,NaN


In [24]:
test_data = retail_event_data.loc[retail_event_data.uid == 1076270].sort_values(by=["time"], ascending=True).reset_index(drop=True)
test_data

,time,uid,event,itemid,trans_id
0,1433221123646,1076270,view,269430,NaN
1,1433221805923,1076270,view,36035,NaN
2,1433221877110,1076270,view,36035,NaN
3,1433221919130,1076270,view,262799,NaN
4,1433221969799,1076270,view,240759,NaN
5,1433222104658,1076270,addtocart,269430,NaN
6,1433222132534,1076270,addtocart,262799,NaN
7,1433222147345,1076270,view,262799,NaN
8,1433222234895,1076270,view,269430,NaN
9,1433222272880,1076270,view,262799,NaN


# CREATE TRAIN, TEST

In [47]:
sorted_filtered_pd_df = pd.read_csv(retail_rocket_root_path + "sorted_filtered_data", header=None, names=["uid", "itemid", "time", "label"])

In [25]:
sorted_filtered_pd_df

,uid,itemid,time,label
0,0,20133,1430802384830,1
1,0,919,1432781848232,1
2,0,4244,1432782322169,1
3,0,0,1433224214164,1
4,0,2921,1433224672007,1
...,...,...,...,...
613690,36750,21591,1438403592429,1
613691,36750,21976,1438403746029,1
613692,36750,27465,1438403820763,1
613693,36750,49707,1438403914087,1


In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import progressbar
import scipy.sparse as sp
spark = SparkSession.builder.appName("create train, test data").getOrCreate()

- Tương tác được sắp xếp theo từng user và theo chiều tăng dần của thời gian
label = 1 -> view;
label = 2 -> add-to-cart; 
label = 3 -> purchase

In [49]:
sorted_filtered_df = spark.read.csv(retail_rocket_root_path + "sorted_filtered_data", "uid INT, itemid INT, time LONG, label INT")
sorted_filtered_df.show()

+---+------+-------------+-----+
|uid|itemid|         time|label|
+---+------+-------------+-----+
|  0| 20133|1430802384830|    1|
|  0|   919|1432781848232|    1|
|  0|  4244|1432782322169|    1|
|  0|     0|1433224214164|    1|
|  0|  2921|1433224672007|    1|
|  0|  5692|1433225555976|    1|
|  0|  6353|1433395087236|    1|
|  0|  6353|1433395158782|    1|
|  0|  6353|1433395205712|    1|
|  0| 10195|1433480149682|    1|
|  0|  9442|1433480383896|    1|
|  0|  1249|1433480524891|    1|
|  0| 10652|1433703062882|    1|
|  0| 10345|1433729380709|    1|
|  0| 10340|1433729489683|    1|
|  0| 10603|1433729786622|    1|
|  0|  9442|1433730019241|    1|
|  0|  8738|1433730231553|    1|
|  0| 13742|1433730402675|    1|
|  0| 17896|1433945313290|    1|
+---+------+-------------+-----+
only showing top 20 rows



In [50]:
print(sorted_filtered_df.count())

613695


- Record: (User, [list of (item, time, label)]) sắp xếp theo time

In [54]:
sorted_filtered_rdd = sorted_filtered_df.rdd.map(lambda x: (x[0], [(x[1], x[2], x[3])])).reduceByKey(lambda v1, v2: v1 + v2)
for i in sorted_filtered_rdd.take(4):
    print(i)

(0, [(20133, 1430802384830, 1), (919, 1432781848232, 1), (4244, 1432782322169, 1), (0, 1433224214164, 1), (2921, 1433224672007, 1), (5692, 1433225555976, 1), (6353, 1433395087236, 1), (6353, 1433395158782, 1), (6353, 1433395205712, 1), (10195, 1433480149682, 1), (9442, 1433480383896, 1), (1249, 1433480524891, 1), (10652, 1433703062882, 1), (10345, 1433729380709, 1), (10340, 1433729489683, 1), (10603, 1433729786622, 1), (9442, 1433730019241, 1), (8738, 1433730231553, 1), (13742, 1433730402675, 1), (17896, 1433945313290, 1), (17428, 1433946626515, 1), (19630, 1434035218038, 1), (20133, 1434035349084, 1), (2679, 1434035926844, 1), (10603, 1434231531446, 1), (29323, 1435014240987, 1), (8664, 1435341565477, 1), (27941, 1436197608867, 1), (6353, 1436503481324, 1), (83046, 1438273381735, 1)])
(4, [(14, 1433221123646, 1), (2504, 1433221805923, 1), (2504, 1433221877110, 1), (4, 1433221919130, 1), (1364, 1433221969799, 1), (14, 1433222104658, 2), (4, 1433222132534, 2), (4, 1433222147345, 1), (14

- Record: (User, list of item, list of label, list of time) sắp xếp theo time

In [55]:
def list_sort(v):
    v.sort(key=lambda e: e[1])
    return v


no_time_filtered_rdd = sorted_filtered_rdd.mapValues(list_sort).mapValues(lambda v: [[e[0] for e in v], [e[2] for e in v], [e[1] for e in v]])
for i in no_time_filtered_rdd.take(1):
    print(i)

(0, [[20133, 919, 4244, 0, 2921, 5692, 6353, 6353, 6353, 10195, 9442, 1249, 10652, 10345, 10340, 10603, 9442, 8738, 13742, 17896, 17428, 19630, 20133, 2679, 10603, 29323, 8664, 27941, 6353, 83046], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1430802384830, 1432781848232, 1432782322169, 1433224214164, 1433224672007, 1433225555976, 1433395087236, 1433395158782, 1433395205712, 1433480149682, 1433480383896, 1433480524891, 1433703062882, 1433729380709, 1433729489683, 1433729786622, 1433730019241, 1433730231553, 1433730402675, 1433945313290, 1433946626515, 1434035218038, 1434035349084, 1434035926844, 1434231531446, 1435014240987, 1435341565477, 1436197608867, 1436503481324, 1438273381735]])


In [56]:
def filter_candidate_users_for_eval(row):
    labels = row[1][1]
    if 2 in labels or 3 in labels:
        return True
    return False

eval_filtered_df = no_time_filtered_rdd.filter(filter_candidate_users_for_eval)
print(eval_filtered_df.count())

7042


In [57]:
def check_test_item(x, window_size):
    user_id = x[0]
    item_list = x[1][0] 
    label_list = x[1][1]
    time_list = x[1][2]
    
    # tính chỉ số của test item, là item có tương tác label=2 or 3 ở thời điểm gần đây nhất
    test_item_id = len(label_list) - 1
    while test_item_id >= 0:
        if label_list[test_item_id] != 1:
            break
        test_item_id -= 1
    return (user_id, [test_item_id, item_list[test_item_id], item_list, label_list, time_list])

check_test_item_df = eval_filtered_df.map(lambda x: check_test_item(x, window_size=5))
for i in check_test_item_df.take(1):
    print(i)

(4, [22, 2504, [14, 2504, 2504, 4, 1364, 14, 4, 4, 14, 4, 2504, 14, 2935, 2932, 14, 2504, 4, 1354, 1372, 2504, 2504, 4, 2504], [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3], [1433221123646, 1433221805923, 1433221877110, 1433221919130, 1433221969799, 1433222104658, 1433222132534, 1433222147345, 1433222234895, 1433222272880, 1433222306083, 1433222323395, 1433222347980, 1433222368861, 1433222449099, 1433222473645, 1433222501643, 1433222666280, 1433222788421, 1433222862175, 1433222876938, 1433223390388, 1433223390388]])


In [58]:
original_neg_df = spark.sparkContext.textFile("explicit.test.negative_in_server")
original_neg_df = original_neg_df.map(lambda x: x.split(")")).map(lambda x: x[0][1:].split(",") + [x[1][1:]]).map(lambda x:(int(x[0]), [int(x[1]), x[2]]))
# original_neg_df = original_neg_df.map(lambda x: x.split(")")).map(lambda x: x[0][1:].split(",")).map(lambda x:(int(x[0]), [int(x[1])]))
for i in original_neg_df.take(1):
    print(i)

(4, [2504, '15383|6979|41741|79116|53192|12932|29099|3562|57164|18277|65827|20751|15666|73745|43855|82476|17937|53721|20929|78537|12905|510|78386|52947|58858|13117|30969|19724|39966|66214|18845|1670|54707|51960|66874|68705|55350|20257|15580|24195|43069|80598|64664|20243|4374|71810|14213|4856|626|16527|27855|69911|4491|37885|78342|59521|33249|43487|77299|80704|73161|23575|16488|16916|6241|47246|58375|16950|17599|31330|12063|5110|42877|49828|44973|69659|43614|76522|73571|5816|35284|78216|47180|21679|18500|32249|9272|71110|36306|41212|520|23968|9871|38286|52557|80291|38163|16118|49313|38768|46231|53357|78822|45425|40796|37600|4000|53134|56947|19542|32312|79032|34579|6354|81460|17238|13512|39708|27612|44617|52359|66406|40072|42703|21531|11621|51837|65263|55740|45018|73312|7756|23249|19770|74465|4632|40235|62644|31763|76556|39195|4652|80992|368|60951|7598|8238|391|57620|43836|29130|60663|64475|22124|67617|32907|72019|35847|27253|34160|75183|78556|69092|41844|60212|54387|1375|20749|60620|403

In [59]:
def check_fail(row):
    v_1 = row[1][0]
    v_2 = row[1][1]
    if v_1[1] != v_2[0]:
        return True
    return False

check_fail_test_item_df = check_test_item_df.join(original_neg_df)
# print(check_fail_test_item_df.count())
# fail_test_item_df = check_fail_test_item_df.filter(check_fail)
# print(fail_test_item_df.count())

# for i in fail_test_item_df.take(1):
#     print(i[0])
#     print(i[1][1][0])
#     print(i[1][0][0])
#     print(i[1][0][1])
#     for idx, j in enumerate(i[1][0][2]):
#         print(idx, j)
#     for idx, j in enumerate(i[1][0][4]):
#         print(idx, j)
#     for idx, j in enumerate(i[1][0][3]):
#         print(idx, j)

def check_weird(x):
    test_item_id = x[1][0][0]
    test_item = x[1][0][1]
    item_list = x[1][0][2]
    label_list = x[1][0][3]
    time_list = x[1][0][4]
    real_test_item = x[1][1][0]
    real_test_item_id = test_item_id
    while item_list[real_test_item_id] != real_test_item:
        if time_list[real_test_item_id] != time_list[test_item_id]:
            return True
        real_test_item_id -= 1
    if time_list[real_test_item_id] != time_list[test_item_id]:
            return True
    return False

def map_weird(x):
    user_id = x[0]
    test_item_id = x[1][0][0]
    test_item = x[1][0][1]
    item_list = x[1][0][2]
    label_list = x[1][0][3]
    time_list = x[1][0][4]
    real_test_item = x[1][1][0]
    real_test_item_id = test_item_id
    while item_list[real_test_item_id] != real_test_item:
        real_test_item_id -= 1
    item_list[real_test_item_id] = test_item
    item_list[test_item_id] = real_test_item
    tmp_label = label_list[real_test_item_id]
    label_list[real_test_item_id] = label_list[test_item_id]
    label_list[test_item_id] = tmp_label
    return (user_id, [test_item_id, item_list[test_item_id], item_list, label_list, time_list])
    
            

# weird_test_item_df = fail_test_item_df.filter(check_weird)
# print(weird_test_item_df.count())
# for i in weird_test_item_df.take(1):
#     print(i)
check_fail_test_item_df = check_fail_test_item_df.map(map_weird)
# fail_test_item_df = check_fail_test_item_df.filter(check_fail)
# print(fail_test_item_df.count())

In [60]:
def filter_explicit_before_test(x):
    user_id = x[0]
    test_item_id = x[1][0]
    test_item = x[1][1]
    item_list = x[1][2]
    label_list = x[1][3]
    time_list = x[1][4]
    check_id = test_item_id - 1
    while check_id >= 0:
        if item_list[check_id] == test_item and label_list[check_id] != 1:
            return True
        check_id -= 1
    return False

filter_ex_df = check_fail_test_item_df.filter(filter_explicit_before_test)
print(filter_ex_df.count())
for i in filter_ex_df.take(1):
    print(i)
    

2759
(180, [19, 2548, [2619, 1536, 2599, 1835, 1538, 1552, 1537, 1516, 847, 152, 317, 1177, 1002, 2065, 640, 2548, 2548, 2548, 2548, 2548, 2548, 1465, 1516, 1516, 1177, 1228, 1511], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1], [1433179811904, 1433179858411, 1433179889415, 1433179920763, 1433179953207, 1433179988697, 1433180016766, 1433180041295, 1433180053494, 1433180065066, 1433180085940, 1433180099587, 1433180125674, 1433180147073, 1433180853320, 1433180878654, 1433180948569, 1433181178489, 1433181204547, 1433184956036, 1433258633392, 1433258740308, 1433258768745, 1433258770326, 1433259063803, 1433259103938, 1433259225545]])


In [61]:
def remove_fake_ex(x):
    user_id = x[0]
    test_item_id = x[1][0]
    test_item = x[1][1]
    item_list = x[1][2]
    label_list = x[1][3]
    time_list = x[1][4]
    check_id = test_item_id - 1
    while check_id >= 0:
        if item_list[check_id] == test_item and label_list[check_id] != 1:
            label_list[check_id] = 1
        check_id -= 1
    return (user_id, [test_item_id, item_list[test_item_id], item_list, label_list, time_list])
final_check_fail_test_item_df = check_fail_test_item_df.map(remove_fake_ex)
filter_ex_df = final_check_fail_test_item_df.filter(filter_explicit_before_test)
print(filter_ex_df.count())
for i in filter_ex_df.take(1):
    print(i)

0


# Create train for retail rocket dataset

In [62]:
import progressbar
import scipy.sparse as sp

def load_interact_matrix(df, num_user, num_item):
    start = datetime.datetime.now()
    # Construct matrix
#     mat = sp.dok_matrix((num_user, num_item), dtype=np.bool_)
    mat = []
    for row in progressbar.ProgressBar()(df.itertuples(index=False)):
        uid, itemid = int(row[0]), int(row[1])
        mat.append((uid, itemid))
    print("time load_interact_matrix: ", datetime.datetime.now() - start)
    return mat

interact_matrix = load_interact_matrix(sorted_filtered_pd_df, len(active_user_list), len(active_item_list))
interact_matrix

| |                  #                           | 613694 Elapsed Time: 0:00:01


time load_interact_matrix:  0:00:01.869515


[(0, 20133),
 (0, 919),
 (0, 4244),
 (0, 0),
 (0, 2921),
 (0, 5692),
 (0, 6353),
 (0, 6353),
 (0, 6353),
 (0, 10195),
 (0, 9442),
 (0, 1249),
 (0, 10652),
 (0, 10345),
 (0, 10340),
 (0, 10603),
 (0, 9442),
 (0, 8738),
 (0, 13742),
 (0, 17896),
 (0, 17428),
 (0, 19630),
 (0, 20133),
 (0, 2679),
 (0, 10603),
 (0, 29323),
 (0, 8664),
 (0, 27941),
 (0, 6353),
 (0, 83046),
 (1, 2406),
 (1, 90),
 (1, 820),
 (1, 1),
 (1, 17),
 (2, 2),
 (2, 1360),
 (2, 1993),
 (2, 836),
 (2, 2929),
 (2, 1362),
 (2, 832),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 35250),
 (3, 25745),
 (3, 7817),
 (3, 25745),
 (3, 5706),
 (3, 46406),
 (3, 64218),
 (3, 6790),
 (3, 1135),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 3),
 (3, 12876),
 (3, 22973),
 (3, 9154),
 (3, 10023),
 (4, 14),
 (4, 2504),
 (4, 2504),
 (4, 4),
 (4, 1364),
 (4, 14),
 (4, 4),
 (4, 4),
 (4, 14),
 (4, 4),
 (4, 2504),
 (4, 14),
 (4, 2935),
 (4, 2932),
 (4, 14),
 (4, 2504),
 (4, 4),
 (4, 1354),
 (4, 1372),
 (4, 2504),
 (4, 2504),
 (4, 4),

In [63]:
interact_matrix_bcast = spark.sparkContext.broadcast(interact_matrix)
interact_matrix_bcast

In [64]:
if (0, 20133) in interact_matrix:
    print(True)

True


In [65]:
import random
def build_train(x, window_size, interact_matrix, num_item):
    
    '''
    Tạo một list gồm s phần tử mà user "uid" chưa tương tác bao giờ
    '''
    def get_neg_list(uid, s):
        neg_list = []
        for _ in range(s):
            j = random.randrange(num_item)
            while (uid, j) in interact_matrix:
                j = random.randrange(num_item)
            neg_list.append(j)
        return neg_list
    
    res = []
    user_id = x[0]
    item_list = x[1][0] 
    label_list = x[1][1] 
    
    # tính chỉ số của test item, là item có tương tác label=2 or 3 ở thời điểm gần đây nhất.   
    test_item_id = len(label_list) - 1
    while test_item_id >= 0:
        if label_list[test_item_id] != 1:
            break
        test_item_id -= 1
        
    # Đối với những record chỉ có label = 1
    if test_item_id < 0:
        for i in range(len(item_list)):
            if i < window_size:
                t = (
                    user_id,
                    [
                     (window_size - i) * [-1] + item_list[:i], 
                     (window_size - i) * [-1] + get_neg_list(user_id, i), 
                     item_list[i],
                     label_list[i]
                    ])
            else:
                t = (
                    user_id, 
                    [
                     item_list[i - window_size:i], 
                     get_neg_list(user_id, window_size), 
                     item_list[i],
                     label_list[i]
                    ])
            res.append(t)
            
    # Đối với những record có label = 2 or 3
    else:
        # Tạo dữ liệu cho tập training, tính đến item ngay trước test item
        for i in range(test_item_id):
            if i < window_size:
                t = (
                    user_id, 
                    [
                     (window_size - i) * [-1] + item_list[:i], 
                     (window_size - i) * [-1] + get_neg_list(user_id, i), 
                     item_list[i],
                     label_list[i]
                    ])
            else:
                t = (
                    user_id, 
                     [
                     item_list[i - window_size:i], 
                     get_neg_list(user_id, window_size), 
                     item_list[i],
                     label_list[i]
                     ])
            res.append(t)
        
        # Tạo dữ liệu cho tập training, tính từ item ngay sau test item
        new_start_point = test_item_id + 1
        for i in range(new_start_point, len(item_list)):
            j = i - new_start_point
            if j < window_size:
                t = (
                    user_id,
                    [
                     (window_size - j) * [-1] + item_list[new_start_point:i], 
                     (window_size - j) * [-1] + get_neg_list(user_id, j), 
                     item_list[i],
                     label_list[i]
                    ])
            else:
                t = (
                    user_id,
                    [
                     item_list[i - window_size:i], 
                     get_neg_list(user_id, window_size), 
                     item_list[i],
                     label_list[i]
                    ])
            res.append(t)
    return res

In [66]:
from pyspark import StorageLevel
def get_real_label(x):
    user_id = x[0]
    if x[1][1] is None:
        item_list = x[1][0][0]
        label_list = x[1][0][1]
        time_list = x[1][0][2]
    else:
        item_list = x[1][1][2]
        label_list = x[1][1][3]
        time_list = x[1][1][4]
    return (user_id, [item_list, label_list, time_list])

num_item = len(active_item_list)
final_train_rdd = no_time_filtered_rdd \
                .leftOuterJoin(final_check_fail_test_item_df) \
                .map(get_real_label) \
                .repartition(8) \
                .persist(StorageLevel.MEMORY_AND_DISK)
                
for m in final_train_rdd.take(5):
    print(m)

(400, [[5051, 65358, 67160, 5920, 32027, 8845, 2142, 913, 68049, 1261, 1261, 770, 3185, 2709, 2091, 2983, 2487, 10413, 77501, 5771], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1430934729695, 1431627278233, 1431974533249, 1431975096134, 1431975122713, 1431975374499, 1431975380753, 1431975382894, 1432140148026, 1432230754385, 1433193949665, 1433194490917, 1433195422865, 1433196555572, 1433198427998, 1433200430433, 1433202002987, 1436979039708, 1437089546631, 1440805375746]])
(420, [[4735, 6333, 4735, 4735, 4735, 4735, 4735, 4735, 1139, 839, 2187, 25745, 7817, 25745, 468, 1470, 917, 917, 917, 78, 78, 78, 823, 332], [1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1432599815427, 1432600053334, 1432600157869, 1432612045174, 1432612088968, 1432614738438, 1432615352983, 1432617131375, 1432930921122, 1432931075716, 1432931162607, 1432931213473, 1432931297422, 1432931618261, 1433135692712, 1433135735240, 1433135884410, 1433135959689, 1433136327592, 

In [67]:
final_train_df = final_train_rdd \
        .flatMap(lambda x: build_train(x, 5, interact_matrix_bcast.value, num_item)) \
        .map(lambda x: (x[0], json.dumps(x[1][0]), json.dumps(x[1][1]), x[1][2], x[1][3])) \
        .toDF(["user_id", "last_item_ids", "last_item_ids_neg", "target_item_id", "target_label"])
final_train_df.show(truncate=False)

+-------+---------------------------------+-----------------------------------+--------------+------------+
|user_id|last_item_ids                    |last_item_ids_neg                  |target_item_id|target_label|
+-------+---------------------------------+-----------------------------------+--------------+------------+
|400    |[-1, -1, -1, -1, -1]             |[-1, -1, -1, -1, -1]               |5051          |1           |
|400    |[-1, -1, -1, -1, 5051]           |[-1, -1, -1, -1, 10172]            |65358         |1           |
|400    |[-1, -1, -1, 5051, 65358]        |[-1, -1, -1, 13401, 19965]         |67160         |1           |
|400    |[-1, -1, 5051, 65358, 67160]     |[-1, -1, 71983, 40638, 38086]      |5920          |1           |
|400    |[-1, 5051, 65358, 67160, 5920]   |[-1, 7771, 19261, 79771, 12166]    |32027         |1           |
|400    |[5051, 65358, 67160, 5920, 32027]|[15897, 69715, 74695, 47714, 41124]|8845          |1           |
|400    |[65358, 67160, 5920

In [68]:
final_train_df.coalesce(1).write.mode("overwrite").csv("dien_train_data", quote="|", quoteAll=True)

# Create test for retail rocket

In [76]:
import json
def build_test(x, window_size):
    user_id = x[0]
    test_item_id = x[1][0]
    test_item = x[1][1]
    item_list = x[1][2]
    label_list = x[1][3]
    time_list = x[1][4]
    
    if test_item_id < window_size:
        t = (
            user_id, 
            [
                (window_size - test_item_id) * [-1] + item_list[:test_item_id],
                item_list[test_item_id],
                label_list[test_item_id]
            ])
    else:
        t = (
            user_id, 
            [
                item_list[test_item_id - window_size:test_item_id], 
                item_list[test_item_id], 
                label_list[test_item_id]
            ])
    return t

In [78]:

final_neg_and_test_df = final_check_fail_test_item_df \
.map(lambda x: build_test(x, window_size=5)).join(original_neg_df) \
.map(lambda x: str(x[0]) + "|" + json.dumps(x[1][0][0]) + "|" + str(x[1][0][1]) + "|" + str(x[1][0][2]) + "|" + x[1][1][1])
for i in final_neg_and_test_df.take(1):
    print(i)

48|[22710, 24834, 24625, 24625, 24625]|19911|2|50597|28293|12634|16223|36861|65307|13843|23822|50778|38072|75421|11317|40895|6705|1419|77773|35077|34054|21990|6619|30407|60015|38121|38062|50912|68290|47435|61459|3320|26958|45870|77623|70376|22583|79575|46329|23341|48696|81196|70219|78906|73866|26082|21998|54840|79350|45330|19973|8384|62433|1891|5365|64636|74085|34167|68723|68079|27630|51193|73911|66537|65010|25212|20317|81696|24296|46159|67453|18981|18817|45200|25151|69064|12009|22773|11276|55940|42788|26108|32017|82411|72219|31752|15724|51591|27600|67113|64706|44369|8746|69275|15452|73738|41784|37373|22466|72728|60866|13291|62230|76660|57672|77494|42170|70426|11607|80587|547|25808|65465|28014|5006|43135|50749|28922|40231|63131|9113|50657|7732|67007|23533|80234|8516|47029|82747|73279|77457|44138|55858|60186|2142|20763|73572|65950|48924|17319|1671|6176|53225|62819|59769|906|40110|51396|32328|16023|22867|29731|80683|76070|39317|7012|17892|35332|16508|8963|17398|2767|60575|29173|13088|564

In [79]:
final_neg_and_test_df.coalesce(1).saveAsTextFile("dien")

In [81]:
t_df = spark.sparkContext.textFile("dien")
print(t_df.count())
for i in t_df.take(1):
    print(i)
    

7042
48|[22710, 24834, 24625, 24625, 24625]|19911|2|50597|28293|12634|16223|36861|65307|13843|23822|50778|38072|75421|11317|40895|6705|1419|77773|35077|34054|21990|6619|30407|60015|38121|38062|50912|68290|47435|61459|3320|26958|45870|77623|70376|22583|79575|46329|23341|48696|81196|70219|78906|73866|26082|21998|54840|79350|45330|19973|8384|62433|1891|5365|64636|74085|34167|68723|68079|27630|51193|73911|66537|65010|25212|20317|81696|24296|46159|67453|18981|18817|45200|25151|69064|12009|22773|11276|55940|42788|26108|32017|82411|72219|31752|15724|51591|27600|67113|64706|44369|8746|69275|15452|73738|41784|37373|22466|72728|60866|13291|62230|76660|57672|77494|42170|70426|11607|80587|547|25808|65465|28014|5006|43135|50749|28922|40231|63131|9113|50657|7732|67007|23533|80234|8516|47029|82747|73279|77457|44138|55858|60186|2142|20763|73572|65950|48924|17319|1671|6176|53225|62819|59769|906|40110|51396|32328|16023|22867|29731|80683|76070|39317|7012|17892|35332|16508|8963|17398|2767|60575|29173|1308

# Note:
    - Mã hóa cho dữ liệu recobell

In [37]:
clean_implicit_data = pd.read_csv(recobell_root_path + '_implicit.clean.txt', header=None, names=['uid', 'itemid', 'time', "label"])

In [38]:
clean_implicit_data.loc[clean_implicit_data.uid == "c6a054f"].sort_values(by=["time"], ascending=True)

,uid,itemid,time,label
3388857,c6a054f,e3f96d5,1470642786,1
3388855,c6a054f,b048538,1470642942,1
3388287,c6a054f,2632f46,1470643059,1
3388856,c6a054f,e1f5cd4,1470643521,1
3388288,c6a054f,6ac9989,1470643600,1


In [39]:
clean_explicit_data = pd.read_csv(recobell_root_path + '_explicit.clean.txt', header=None, names=['uid', 'itemid', 'time', "label"])

In [40]:
clean_explicit_data

,uid,itemid,time,label
0,7e525ea,430ae4b,1470794116,2
1,c6a054f,b048538,1470643815,2
2,c6a054f,e1f5cd4,1470643815,2
3,c6a054f,c6df445,1470643815,2
4,1afa4ea,b1c6d74,1470989514,2
...,...,...,...,...
360501,578675f,90c1690,1471048488,2
360502,5f41838,74f0d91,1470997994,2
360503,101b0d6,a7881b3,1470722569,2
360504,40b4a5a,c05af1d,1470716292,2


In [41]:
clean_explicit_data.loc[clean_explicit_data.uid == "c6a054f"].sort_values(by=["time"], ascending=True)

,uid,itemid,time,label
1,c6a054f,b048538,1470643815,2
2,c6a054f,e1f5cd4,1470643815,2
3,c6a054f,c6df445,1470643815,2
15,c6a054f,1a0d6aa,1470643815,2
16,c6a054f,a203c8c,1470643815,2


In [17]:
with open(recobell_root_path + 'i2index.txt') as f:
    active_item_dict = {str(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_item_list = active_item_dict.keys()
print(len(active_item_list))

118293


In [18]:
with open(recobell_root_path + 'u2index.txt') as f:
    active_user_dict = {str(line.split(',')[0]): int(line.split(',')[1]) for line in f}
active_user_list = active_user_dict.keys()
print(len(active_user_list))

206203


In [19]:
print("Length before filter: ", len(clean_implicit_data))
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
clean_implicit_data = clean_implicit_data[clean_implicit_data.uid.isin(active_user_list)]
print("length after filter user: ", len(clean_implicit_data))
clean_implicit_data = clean_implicit_data[clean_implicit_data.itemid.isin(active_item_list)]
print("length after filter item: ", len(clean_implicit_data))

Length before filter:  4521505
length after filter user:  3401237
length after filter item:  3401237


In [20]:
print("Length before filter: ", len(clean_explicit_data))
# loai bo tuong tac ma user, item khong co trong active_user_list va active_item_list.
clean_explicit_data = clean_explicit_data[clean_explicit_data.uid.isin(active_user_list)]
print("length after filter user: ", len(clean_explicit_data))
clean_explicit_data = clean_explicit_data[clean_explicit_data.itemid.isin(active_item_list)]
print("length after filter item: ", len(clean_explicit_data))

Length before filter:  360506
length after filter user:  157321
length after filter item:  157264


In [23]:
filter_implicit_data = clean_implicit_data.apply(lambda x: pd.Series([active_user_dict[x['uid']], active_item_dict[x['itemid']], x['time'], x["label"]], index=["uid", "itemid", "time", "label"]), axis=1)

In [ ]:
filter_explicit_data = clean_explicit_data.apply(lambda x: pd.Series([active_user_dict[x['uid']], active_item_dict[x['itemid']], x['time'], x["label"]], index=["uid", "itemid", "time", "label"]), axis=1)

In [24]:
filter_implicit_data.to_csv(recobell_root_path + "_implicit.filter.clean.txt", header=False, index=False)

In [25]:
filter_explicit_data.to_csv(recobell_root_path + "_explicit.filter.clean.txt", header=False, index=False)

In [4]:
filter_implicit_data = pd.read_csv(recobell_root_path + "_implicit.filter.clean.txt", header=None, names=['uid', 'itemid', 'time', "label"])
filter_implicit_data

,uid,itemid,time,label
0,0,0,1470917081,1
1,0,1,1470916631,1
2,0,2,1470844977,1
3,0,3,1470811668,1
4,0,4,1470811985,1
5,0,5,1470701526,1
6,1,6,1471021005,1
7,1,7,1470895701,1
8,1,7,1470895586,1
9,1,8,1470843750,1


In [5]:
filter_explicit_data = pd.read_csv(recobell_root_path + "_explicit.filter.clean.txt", header=None, names=['uid', 'itemid', 'time', "label"])
filter_explicit_data

,uid,itemid,time,label
0,154592,2356,1470643815,2
1,154592,11345,1470643815,2
2,154592,2669,1470643815,2
3,154595,114,1470989514,2
4,4,28,1470710071,2
5,90212,42,1471092013,2
6,154604,401,1470975642,2
7,90221,929,1470645412,2
8,90209,659,1470631213,2
9,12674,17150,1471183701,2


In [6]:
filtered_data = filter_implicit_data.append(filter_explicit_data)
filtered_data

,uid,itemid,time,label
0,0,0,1470917081,1
1,0,1,1470916631,1
2,0,2,1470844977,1
3,0,3,1470811668,1
4,0,4,1470811985,1
5,0,5,1470701526,1
6,1,6,1471021005,1
7,1,7,1470895701,1
8,1,7,1470895586,1
9,1,8,1470843750,1


In [7]:
sorted_filtered_pd_df = filtered_data.sort_values(by=["uid", "time"], ascending=True).reset_index(drop=True)
sorted_filtered_pd_df

,uid,itemid,time,label
0,0,15,1470701505,1
1,0,5,1470701526,1
2,0,5,1470701744,1
3,0,3,1470811668,1
4,0,4,1470811673,1
5,0,14,1470811765,1
6,0,4,1470811985,1
7,0,2,1470844977,1
8,0,13,1470845413,1
9,0,1,1470916631,1


In [9]:
sorted_filtered_pd_df.to_csv(recobell_root_path + "sorted_filtered_data", header=False, index=False)

# create train, test

In [5]:
sorted_filtered_pd_df = pd.read_csv(retail_rocket_root_path + "sorted_filtered_data", header=None, names=["uid", "itemid", "time", "label"])

In [28]:
sorted_filtered_pd_df

,uid,itemid,time,label
0,0,20133,1430802384830,1
1,0,919,1432781848232,1
2,0,4244,1432782322169,1
3,0,0,1433224214164,1
4,0,2921,1433224672007,1
5,0,5692,1433225555976,1
6,0,6353,1433395087236,1
7,0,6353,1433395158782,1
8,0,6353,1433395205712,1
9,0,10195,1433480149682,1


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [9]:
spark = SparkSession.builder.appName("create train, test data").getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

In [16]:
sorted_filtered_df = spark.read.csv(retail_rocket_root_path + "sorted_filtered_data", "uid INT, itemid INT, time LONG, label INT")
sorted_filtered_df.show()

+---+------+-------------+-----+
|uid|itemid|         time|label|
+---+------+-------------+-----+
|  0| 20133|1430802384830|    1|
|  0|   919|1432781848232|    1|
|  0|  4244|1432782322169|    1|
|  0|     0|1433224214164|    1|
|  0|  2921|1433224672007|    1|
|  0|  5692|1433225555976|    1|
|  0|  6353|1433395087236|    1|
|  0|  6353|1433395158782|    1|
|  0|  6353|1433395205712|    1|
|  0| 10195|1433480149682|    1|
|  0|  9442|1433480383896|    1|
|  0|  1249|1433480524891|    1|
|  0| 10652|1433703062882|    1|
|  0| 10345|1433729380709|    1|
|  0| 10340|1433729489683|    1|
|  0| 10603|1433729786622|    1|
|  0|  9442|1433730019241|    1|
|  0|  8738|1433730231553|    1|
|  0| 13742|1433730402675|    1|
|  0| 17896|1433945313290|    1|
+---+------+-------------+-----+
only showing top 20 rows



In [17]:
print(sorted_filtered_df.count())

613695


In [171]:
sorted_filtered_df = sorted_filtered_df.rdd.map(lambda x: (x[0], [(x[1], x[2], x[3])])).reduceByKey(lambda v1, v2: v1 + v2)
print(sorted_filtered_df.take(4))

[(0, [(20133, 1430802384830, 1), (919, 1432781848232, 1), (4244, 1432782322169, 1), (0, 1433224214164, 1), (2921, 1433224672007, 1), (5692, 1433225555976, 1), (6353, 1433395087236, 1), (6353, 1433395158782, 1), (6353, 1433395205712, 1), (10195, 1433480149682, 1), (9442, 1433480383896, 1), (1249, 1433480524891, 1), (10652, 1433703062882, 1), (10345, 1433729380709, 1), (10340, 1433729489683, 1), (10603, 1433729786622, 1), (9442, 1433730019241, 1), (8738, 1433730231553, 1), (13742, 1433730402675, 1), (17896, 1433945313290, 1), (17428, 1433946626515, 1), (19630, 1434035218038, 1), (20133, 1434035349084, 1), (2679, 1434035926844, 1), (10603, 1434231531446, 1), (29323, 1435014240987, 1), (8664, 1435341565477, 1), (27941, 1436197608867, 1), (6353, 1436503481324, 1), (83046, 1438273381735, 1)]), (4, [(14, 1433221123646, 1), (2504, 1433221805923, 1), (2504, 1433221877110, 1), (4, 1433221919130, 1), (1364, 1433221969799, 1), (14, 1433222104658, 2), (4, 1433222132534, 2), (4, 1433222147345, 1), (

In [40]:
def list_sort(v):
    v.sort(key=lambda e: e[1])
    return v


no_time_filtered_rdd = sorted_filtered_df.mapValues(list_sort).mapValues(lambda v: [[e[0] for e in v], [e[2] for e in v]])
print(no_time_filtered_rdd.take(1))

[(0, [[20133, 919, 4244, 0, 2921, 5692, 6353, 6353, 6353, 10195, 9442, 1249, 10652, 10345, 10340, 10603, 9442, 8738, 13742, 17896, 17428, 19630, 20133, 2679, 10603, 29323, 8664, 27941, 6353, 83046], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])]


In [133]:
import progressbar
import scipy.sparse as sp

def load_interact_matrix(df, num_user, num_item):
    start = datetime.datetime.now()
    # Construct matrix
#     mat = sp.dok_matrix((num_user, num_item), dtype=np.bool_)
    mat = []
    for row in progressbar.ProgressBar()(df.itertuples(index=False)):
        uid, itemid = int(row[0]), int(row[1])
        mat.append((uid, itemid))
    print("time load_interact_matrix: ", datetime.datetime.now() - start)
    return mat

interact_matrix = load_interact_matrix(sorted_filtered_pd_df, len(active_user_list), len(active_item_list))
interact_matrix

| |             #                                | 613694 Elapsed Time: 0:00:01


time load_interact_matrix:  0:00:01.423218


[(0, 20133),
 (0, 919),
 (0, 4244),
 (0, 0),
 (0, 2921),
 (0, 5692),
 (0, 6353),
 (0, 6353),
 (0, 6353),
 (0, 10195),
 (0, 9442),
 (0, 1249),
 (0, 10652),
 (0, 10345),
 (0, 10340),
 (0, 10603),
 (0, 9442),
 (0, 8738),
 (0, 13742),
 (0, 17896),
 (0, 17428),
 (0, 19630),
 (0, 20133),
 (0, 2679),
 (0, 10603),
 (0, 29323),
 (0, 8664),
 (0, 27941),
 (0, 6353),
 (0, 83046),
 (1, 2406),
 (1, 90),
 (1, 820),
 (1, 1),
 (1, 17),
 (2, 2),
 (2, 1360),
 (2, 1993),
 (2, 836),
 (2, 2929),
 (2, 1362),
 (2, 832),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 35250),
 (3, 25745),
 (3, 7817),
 (3, 25745),
 (3, 5706),
 (3, 46406),
 (3, 64218),
 (3, 6790),
 (3, 1135),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 28),
 (3, 3),
 (3, 12876),
 (3, 22973),
 (3, 9154),
 (3, 10023),
 (4, 14),
 (4, 2504),
 (4, 2504),
 (4, 4),
 (4, 1364),
 (4, 14),
 (4, 4),
 (4, 4),
 (4, 14),
 (4, 4),
 (4, 2504),
 (4, 14),
 (4, 2935),
 (4, 2932),
 (4, 14),
 (4, 2504),
 (4, 4),
 (4, 1354),
 (4, 1372),
 (4, 2504),
 (4, 2504),
 (4, 4),

In [125]:
interact_matrix = [[0, 20133]]

In [134]:
interact_matrix_bcast = spark.sparkContext.broadcast(interact_matrix)
interact_matrix_bcast

In [136]:
if (0, 20133) in interact_matrix:
    print(True)

True


In [179]:
import random
def build_train(x, window_size, interact_matrix, num_item):
    def get_neg_list(uid, s):
        get_neg_list = []
        for _ in range(s):
            j = random.randrange(num_item)
            while (uid, j) in interact_matrix:
                j = random.randrange(num_item)
            get_neg_list.append(j)
        return get_neg_list
    
    res = []
    user_id = x[0]
    item_list = x[1][0] 
    label_list = x[1][1] 
    test_index = len(label_list) - 1
    while test_index >= 0:
        if label_list[test_index] != 1:
            break
        test_index -= 1
    # khong co addtocart va purchase
    if test_index < 0:
        # for training
        for i in range(len(item_list)):
            if i < window_size:
                t = [True,
                     user_id, 
                     (window_size - i) * [-1] + item_list[:i], 
                     (window_size - i) * [-1] + get_neg_list(user_id, i), 
                     item_list[i],
                     label_list[i],
                    ]
            else:
                t = [True,
                     user_id, 
                     item_list[i - window_size:i], 
                     get_neg_list(user_id, window_size), 
                     item_list[i],
                     label_list[i]
                    ]
            res.append(t)
            
    # co addtocart hoac purchase
    else:
        # for training
        for i in range(test_index):
            if i < window_size:
                t = [True,
                     user_id, 
                     (window_size - i) * [-1] + item_list[:i], 
                     (window_size - i) * [-1] + get_neg_list(user_id, i), 
                     item_list[i],
                     label_list[i]
                    ]
            else:
                t = [True,
                     user_id, 
                     item_list[i - window_size:i], 
                     get_neg_list(user_id, window_size), 
                     item_list[i],
                     label_list[i]
                     ]
            res.append(t)
        
        # for training
        new_start_point = test_index + 1
        for i in range(new_start_point, len(item_list)):
            if i - new_start_point < window_size:
                t = [True,
                     user_id, 
                     (window_size - i) * [-1] + item_list[new_start_point:i], 
                     (window_size - i) * [-1] + get_neg_list(user_id, i - new_start_point), 
                     item_list[i],
                     label_list[i]
                    ]
            else:
                t = [True,
                     user_id, 
                     item_list[i - window_size:i], 
                     get_neg_list(user_id, window_size), 
                     item_list[i],
                     label_list[i]
                     ]
                res.append(t)
        
        # for testing
        if test_index < window_size:
            t = [False,
                 user_id, 
                 (window_size - test_index) * [-1] + item_list[:test_index],
                 item_list[test_index],
                 label_list[test_index]
                ]
        else:
            t = [False,
                 user_id, 
                 item_list[test_index - window_size:test_index],
                 item_list[test_index],
                 label_list[test_index]
                ]
        res.append(t)

#     # for training
#     for i in range(len(item_list)):
#         if i < window_size:
#             t = [True,
#                  user_id, 
#                  (window_size - i) * [-1] + item_list[:i], 
#                  (window_size - i) * [-1] + get_neg_list(user_id, i), 
#                  item_list[i],
#                  label_list[i]
#                 ]
#         else:
#             t = [True,
#                  user_id, 
#                  item_list[i - window_size:i], 
#                  get_neg_list(user_id, window_size), 
#                  item_list[i],
#                  label_list[i]
#                 ]
#         res.append(t)
    return res

In [180]:
test_final_df = no_time_filtered_rdd
# .filter(lambda x: x[0] == 4)
print(test_final_df.take(5))

[(0, [[20133, 919, 4244, 0, 2921, 5692, 6353, 6353, 6353, 10195, 9442, 1249, 10652, 10345, 10340, 10603, 9442, 8738, 13742, 17896, 17428, 19630, 20133, 2679, 10603, 29323, 8664, 27941, 6353, 83046], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), (4, [[14, 2504, 2504, 4, 1364, 14, 4, 4, 14, 4, 2504, 14, 2935, 2932, 14, 2504, 4, 1354, 1372, 2504, 2504, 4, 2504], [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3]]), (8, [[8, 8, 7008, 8724, 8452, 9132], [1, 1, 1, 1, 1, 1]]), (12, [[2922, 1367, 12, 1743, 21, 1992, 1367, 827, 19, 13, 25184, 2479, 41830, 531, 18939], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), (16, [[392, 1658, 2529, 1359, 1352, 1982, 1983, 20], [1, 1, 1, 1, 1, 1, 1, 1]])]


In [181]:
num_item = len(active_item_list)
final_df = test_final_df.flatMap(lambda x: build_train(x, 5, interact_matrix_bcast.value, num_item))
# for m in final_df.collect():
#     print(m)

In [182]:
import json
train_data_rdd = final_df.filter(lambda x: x[0]).map(lambda x: [x[1], json.dumps(x[2]), json.dumps(x[3]), x[4], x[5]])

In [187]:
train_data_df = train_data_rdd.toDF(["user_id", "last_item_ids", "last_item_ids_neg", "target_item_id", "target_label"])
train_data_df.show(truncate=False)

+-------+----------------------------------+-----------------------------------+--------------+------------+
|user_id|last_item_ids                     |last_item_ids_neg                  |target_item_id|target_label|
+-------+----------------------------------+-----------------------------------+--------------+------------+
|0      |[-1, -1, -1, -1, -1]              |[-1, -1, -1, -1, -1]               |20133         |1           |
|0      |[-1, -1, -1, -1, 20133]           |[-1, -1, -1, -1, 39573]            |919           |1           |
|0      |[-1, -1, -1, 20133, 919]          |[-1, -1, -1, 45531, 42862]         |4244          |1           |
|0      |[-1, -1, 20133, 919, 4244]        |[-1, -1, 64557, 26978, 28892]      |0             |1           |
|0      |[-1, 20133, 919, 4244, 0]         |[-1, 36430, 2946, 53740, 4764]     |2921          |1           |
|0      |[20133, 919, 4244, 0, 2921]       |[71005, 17569, 40505, 22003, 78312]|5692          |1           |
|0      |[919, 4244

In [190]:
train_data_df.write.mode("overwrite").csv(retail_rocket_root_path + "sequence_train_data", quote="|", quoteAll=True)